In [ ]:
#Install necessary libraries
!pip install pandas pymongo langchain langchain_community llama-cpp-python llama-index llama-index-embeddings-huggingface


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install requests


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#Step 1 : CSV Data Management
# Load CSV data into MongoDB
import pandas as pd
from pymongo import MongoClient

#Function to load the sample data to the Nosql DB
def load_csv_mongodb(path):
    df = pd.read_csv(path)
    #data cleaning
    df['Discount'] = df['Discount'].str.replace('%', '').astype(float)
    df['LaunchDate'] = pd.to_datetime(df['LaunchDate'], dayfirst=True, errors='coerce')
    print(df.info())
    print(df)
    #provide the necessary client,db,and collection names
    client = MongoClient('your_mongo_uri') #replace with your mongo url
    db = client['AIQoD']
    collection = db['Products']
    #delete and insert into collections
    collection.delete_many({})
    collection.insert_many(df.to_dict(orient='records'))
    client.close()
    
    print(f"Inserted {len(df)} documents into MongoDB.")
    return



In [ ]:
#Step 2: Dynamic Query Generation using LLM
import requests
import json
import ast
#Use llama3.2 to generate a NoSql query givne the user input

def generate_query(user_query,path):
    

    df = pd.read_csv(path)
    query = f"""
    Write a python MongoDB query in the exact format: db.collection.find({{ "condition":{{"field": {{ "operator": value }}}} }})
    Find documents where: {user_query}
    Output ONLY the complete db.collection.find() statement with nothing else.
    Note: {df.columns} are  the column names
    """


    data = {
        "model": "llama3.2",
        "prompt": query,
        "stream": True  # This is the default, but just to be explicit
    }

    response = requests.post("http://localhost:11434/api/generate", json=data, stream=True)

    # Read and decode line-by-line (each line is a JSON object)
    generated_text = ""
    for line in response.iter_lines():
        if line:
            part = json.loads(line.decode("utf-8"))
            generated_text += part.get("response", "")


    print(generated_text)

    #Strip the data and create a dictionary for quering collections
     
    try:
        query_start = generated_text.find("find(") + len("find(")
        query_end = generated_text.rfind(")")
        query_str = generated_text[query_start:query_end].strip()
        #query_dict = ast.literal_eval(query_str)
        query_dict = json.loads(query_str)
        print(query_dict)
    except Exception as e:
        print(f"Error parsing query: {e}")
        query_dict = {}
    print(query_dict)

    return query_dict,generated_text


In [ ]:
from llama_index.core import Document
from pymongo.mongo_client import MongoClient
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import StorageContext
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings

#provide the mongourl,db,collection as needed
def llama_index_mongodb(query_dict,user_query,path,response):
    df = pd.read_csv(path)#give the file path
    # Create a MongoClient
    client = MongoClient('your_mongo_uri')

    # Select the database and collection
    db = client['AIQoD'] #enter your db name
    collection = db['Products'] #enter your collections name

    # Get all documents (cursor is exhausted after first iteration)
    documents = list(collection.find({}))
    print(f'Loaded {len(documents)} documents')

    Settings.llm = Ollama(
        model="llama3.2",  
        base_url="http://localhost:11434",  
        request_timeout=60.0
    )

    embed_model = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")
    Settings.embed_model = embed_model
    
    
    # Execute query and convert to DataFrame immediately (cursor exhausts after iteration)
    #results = list(collection.find(query_dict))

    # Convert to LlamaIndex Document format
    documents = [
        Document(text=str(doc))  # You can customize how the doc is formatted
        for doc in documents
    ]

    #print(documents)

    index = VectorStoreIndex.from_documents(documents)

    # Run a query
    query_engine = index.as_query_engine()
    results = query_engine.query(response)
    #print(results)


    #print the results if any matching records found in collections using regular quring method
    if results:
        df=pd.DataFrame(results)
        df = df.drop(df.columns[0], axis=1)
        # Ask the user what they want to do
        print("\n What would you like to do with the retrieved data?")
        print("1. Display the data")
        print("2. Save the data as CSV")
        choice = input("Enter 1 or 2: ")

        if choice == "1":
            print("\nRetrieved Data:\n")
            print(df)

        elif choice == "2":
            test_case_number = input("Enter test case number (e.g., 1 for test_case1.csv): ")
            filename = f"test_case{test_case_number}.csv"
            df.to_csv(filename, index=False)
            print(f"\n Data saved to: {filename}")

        else:
            print(" Invalid choice. Please enter 1 or 2.")

    #Setup indeces for each document  
    llama_documents=[]
    fields_to_include=['ProductID','ProductName','Category','Price','Rating','ReviewCount','Stock','Discount'
                   ,'Brand','LaunchDate']

    for doc in documents:
        metadata = {key: doc.get(key, None) for key in fields_to_include}
        llama_doc=Document(text="",metadata=metadata)
        llama_documents.append(llama_doc)

    
    # Ensure you have the MongoDB URI with appropriate credentials
    mongo_uri = 'you_mongo_uri' #give your uri
    mongodb_client = MongoClient(mongo_uri,    serverSelectionTimeoutMS=30000,  # 30 seconds for server selection
        socketTimeoutMS=60000,          # 60 seconds for socket operations
        connectTimeoutMS=30000,         # 30 seconds for connection
        waitQueueTimeoutMS=30000        # 30 seconds to wait for connection from pool
        )

    # Create an instance of MongoDBAtlasVectorSearch, vector index name should match that of the one created in mongoatlas
    vector_store = MongoDBAtlasVectorSearch(mongodb_client,    
                                            db_name="AIQoD",
                                            collection_name="Products",
                                            vector_index_name="vector_index" 
                                        

    )

    # now create an index from all the Documents and store them in Atlas
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex.from_documents(
        llama_documents, storage_context=storage_context,
        show_progress=True, # this will show you a progress bar as the embeddings are created
    )
    # Create a VectorStoreIndex from the MongoDBAtlasVectorSearch instance
    index = VectorStoreIndex.from_vector_store(vector_store)

    query_engine = index.as_query_engine(
        vector_store_kwargs={
            "filter": query_dict,
            "search_kwargs": {
                "k": 10000,  # Number of results to return
                "score_threshold": 0  # Minimum similarity score
            }
        },
        response_mode="compact",      # Optional: makes output easier to parse
        return_source=True
    )
    result = query_engine.query(user_query)
    print(result)
    client.close()
    return 



In [ ]:
#main program to call the other functions 
path=r'..\AIQoD\sample_data.csv'
load_csv_mongodb(path)
user_query = input('Enter your query : ')
query_dictionary,response= generate_query(user_query,path)
llama_index_mongodb(query_dictionary,user_query,path,response)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ProductID    10 non-null     int64         
 1   ProductName  10 non-null     object        
 2   Category     10 non-null     object        
 3   Price        10 non-null     float64       
 4   Rating       10 non-null     float64       
 5   ReviewCount  10 non-null     int64         
 6   Stock        10 non-null     int64         
 7   Discount     10 non-null     float64       
 8   Brand        10 non-null     object        
 9   LaunchDate   10 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(3), object(3)
memory usage: 932.0+ bytes
None
   ProductID                  ProductName        Category   Price  Rating  \
0        101               Wireless Mouse     Electronics   25.99     4.5   
1        102              Gaming Keyboard     Electronics   75.49

Generating embeddings: 100%|██████████| 10/10 [00:00<00:00, 38.38it/s]


Empty Response
